In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
import time
import datetime

In [10]:
driver = webdriver.Chrome()

In [11]:
driver.get('https://www.linkedin.com/login')
email_input = driver.find_element(By.ID, 'username')
password_input = driver.find_element(By.ID, 'password')
email_input.send_keys('hammad173216@gmail.com')
password_input.send_keys('@hammad40099257')
password_input.send_keys(Keys.ENTER)

In [12]:
url='https://www.linkedin.com/jobs/search/?currentJobId=3703004786&geoId=101022442&location=Pakistan'

In [13]:
driver.get(url)

In [14]:
def scrape_job_links(page):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    x=datetime.datetime.now().strftime("%d-%m-%Y--%I-%M-%p")
    with open(f'html_content__{x}__page{page}.txt', 'w',encoding='utf-8') as f:
        f.write(soup.prettify())
    job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})
    if len(job_postings)!=0:
        data = []
        for job_posting in job_postings:
            try:
                job_title = job_posting.find('a', class_='job-card-list__title').get_text().strip()
            except AttributeError:
                job_title = None
            try:
                job_id = job_posting.get("data-occludable-job-id")
            except AttributeError:
                job_id = None
            try:
                Job_link = job_posting.find('a', class_='job-card-list__title').get("href")
                Job_link= 'https://www.linkedin.com'+Job_link
            except AttributeError:
                Job_link = None
                
            data.append({
                    'Job Title': job_title,
                    'Job ID' :job_id,
                    'Job Link':Job_link
                    })
        
        df = pd.DataFrame(data)
        return df
    else:
        return None


In [15]:
def save_scraper_note(starting_time,ending_time,total_jobs,total_pages):
    x=datetime.datetime.now().strftime("%d-%m-%Y--%I-%M-%p")
    with open(f'notes/Scrapper_note__{x}.txt', 'w',encoding='utf-8') as f:
        f.write(f'Starting Time:{starting_time}\nEnding Time:{ending_time}\nTotal Jobs:{total_jobs}\nTotal Pages:{total_pages}')
    

In [16]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Assuming you have imported the necessary modules and initialized your webdriver as 'driver'
current_page_number=1
total_jobs=0
starting_time=datetime.datetime.now().strftime("%d-%m-%Y--%I-%M-%p-%S")
total_pages=0
while(True):
    time.sleep(10)
    data=scrape_job_links(current_page_number)
    if data is not None:
        x=datetime.datetime.now().strftime("%d-%m-%Y--%I-%M-%p")
        y=len(data['Job ID'].unique())
        data.to_csv(f'linkedin_jobs_page-{current_page_number}__{x}__{y}.csv', index=False)
        total_pages=current_page_number
        total_jobs=total_jobs+y
        
    time.sleep(2)
    current_page_number=current_page_number+1
    next_button_xpath = f"//button[@aria-label='Page {current_page_number}']"
    try:
        next_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
        next_button.click()
    except TimeoutException:
        print(f"Next button for page {current_page_number} not found. End of pages.")
        break
ending_time=datetime.datetime.now().strftime("%d-%m-%Y--%I-%M-%p-%S")
save_scraper_note(starting_time,ending_time,total_jobs,total_pages)
print(f'starting time of scrapper:{starting_time}')
print(f'Ending time of scrapper:{ending_time}')
print(f'Total Jobs Scrapped:{total_jobs}')


Next button for page 35 not found. End of pages.
starting time of scrapper:01-12-2023--09-41-AM-21
Ending time of scrapper:01-12-2023--09-49-AM-20
Total Jobs Scrapped:780
